In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import h5py
import cv2
import os
import glob
import scipy
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import Adadelta, Adam
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, History
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import pandas as pd
from __future__ import division, print_function
from collections import Counter
import ujson as json
import PIL
import random

from collections import defaultdict

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

In [ ]:
# paths
data_path = DATA_HOME_DIR + '/' 
full_train_path = data_path + 'train_org/'
crop_path = data_path + 'cropped/'
split_train_path = data_path + 'train/'
valid_path = ROOT_DIR + '/valid/data/train/'
test_path = DATA_HOME_DIR + '/test2/'

# data
classes = ["ALB", "BET", "DOL", "LAG", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

In [ ]:
class_dict = defaultdict(str)

for fp in glob.glob(full_train_path + '*/*g'):
    cls = fp.split('/')[-2]
    im = fp.split('/')[-1]
    class_dict[im] = cls
    
print("Image Records:", len(class_dict.keys()))

In [ ]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224),classes=None):
    return gen.flow_from_directory(dirname, target_size=target_size, class_mode=class_mode, shuffle=shuffle, batch_size=batch_size,classes=classes)

In [ ]:
def get_classes(trn_path, val_path, test_path):
    batches = get_batches(trn_path, shuffle=False, batch_size=1, target_size=(224,224))
    val_batches = get_batches(val_path, shuffle=False, batch_size=1, target_size=(224,224))
    test_batches = get_batches(test_path, shuffle=False, batch_size=1, target_size=(224,224))
    return val_batches.filenames, batches.filenames, test_batches.filenames

In [ ]:
val_filenames, filenames, test_filenames = get_classes(split_train_path, valid_path, test_path)

In [ ]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [ ]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']
bb_json = {}

for c in anno_classes:
    j = json.load(open('bb_annotations/{}.json'.format(c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [ ]:
fns = glob.glob(full_train_path + '*/*.jpg')
    
for fn in fns:
    f_id = fn.split("/")[-1]
    im = PIL.Image.open(fn)
    width, height = im.size
    
    if not f_id in bb_json.keys():
        bb_json[f_id] = {"height": 0, "width": 0, "x": 0, "y": 0, "size": im.size}
    else:
        anno = bb_json[f_id]
        x, y = anno["x"], anno["y"]
        bb_json[f_id] = {"height": anno["height"], "width": anno["width"], "x": x, "y": y, "size": im.size}

In [ ]:

bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb,resized_width=224.,resized_height=224.):
    cropsize = 224
    size = bb["size"]
    bb = [bb[p] for p in bb_params]
    
    # conversion factors
    conv_x = (resized_width / size[1])
    conv_y = (resized_height / size[0])
    
    # make the size conversions
    height = bb[0]*conv_x
    width = bb[1]*conv_y
    x = bb[2]*conv_y
    y = bb[3]*conv_x
    
    # offset/padding adjustments
    x = max(x , 0)
    y = max(y , 0)
    
#     if x + cropsize > width:
#         x = width - cropsize
#     if y + cropsize > height:
#         y = height - cropsize
    
    bb[0] = int(height)
    bb[1] = int(width)
    bb[2] = int(x)
    bb[3] = int(y)
    return bb

In [ ]:
trn_bbox = np.stack([convert_bb(bb_json[f]) for f in raw_filenames]).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f]) for f in raw_val_filenames]).astype(np.float32)

In [ ]:
def get_data(path, target_size=(360,640)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

In [ ]:
def get_category(path, target_size=(1,1)):
    batches = get_batches(path, shuffle=False, batch_size=1,classes=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], target_size=target_size)
    return np.concatenate([batches.next()[1] for i in range(batches.nb_sample)])

In [ ]:
smp_class = get_category(valid_path)
trn_class = get_category(split_train_path)

In [ ]:
smp = get_data(valid_path,target_size=(1,1))
trn = get_data(split_train_path,target_size=(1,1))

# smp = get_data(valid_path,target_size=(360,640))
# trn = get_data(split_train_path,target_size=(360,640))

In [ ]:
def create_rect(bb, color='red'):
    return plt.Rectangle((bb[2], bb[3]), bb[1], bb[0], color=color, fill=False, lw=3)

def show_bb(i):
    bb = trn_bbox[i]
    plot(trn[i])
    plt.gca().add_patch(create_rect(bb))

In [ ]:
def to_plot(img):
    return np.rollaxis(img, 0, 3).astype(np.uint8)

def plot(img):
    plt.imshow(to_plot(img))

In [ ]:
def get_coordinates(bbox):
    height = bbox[0]
    width = bbox[1]
    
    #bottom left
    x0 = bbox[2] 
    y0 =  bbox[3]

    return [[x0,y0],[x0+width,y0],[x0+width,y0+height],[x0,y0+height]]

In [ ]:
# polygon_test = []

# polygon_test.append(np.array(get_coordinates(val_bbox[14]),np.int32))
# # polygon_test

# mask = np.zeros((360, 640), dtype="int32")
# mask = cv2.fillPoly(mask, np.int32(polygon_test),255)
# plt.imshow(mask)

In [ ]:
trn_bbox = [convert_bb(bb_json[f],resized_width=22,resized_height=40) for f in raw_filenames]
val_bbox = [convert_bb(bb_json[f],resized_width=22,resized_height=40) for f in raw_val_filenames]

In [ ]:
trn_targets = []
val_targets = [] 


#training bboxes
for i,b in enumerate(trn_bbox):
    
    #get individual fish bboxes
    mask_patches = []
    mask_patches.append(np.array(get_coordinates(trn_bbox[i]),np.int32))    
    
    
    mask = np.zeros((22, 40), dtype="int32")
    
    mask = cv2.fillPoly(mask, np.int32(mask_patches),255)
    trn_targets.append(mask)
    
    
#validation bboxes
for i,b in enumerate(val_bbox):
    
    mask_patches = []
    mask_patches.append(np.array(get_coordinates(val_bbox[i]),np.int32))
        
        
    mask = np.zeros((22, 40), dtype="int32")
    mask = cv2.fillPoly(mask, np.int32(mask_patches),255)
    val_targets.append(mask)



In [ ]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

def VGG_16(size=(224, 224), weights_path='data/vgg16_bn_conv.txt'):
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,)+size))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))


    if weights_path:
        model.load_weights(weights_path)

    return model

# Mask based model:

In [ ]:
nf=128; p=0.6

size = (360, 640)
model = VGG_16(size=size)


for layer in model.layers:
    layer.trainable = False

model.pop()

model.add(BatchNormalization(axis=1))
model.add(Convolution2D(nf,3,3, activation='relu', border_mode='same'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(p/3))

model.add(Convolution2D(nf,3,3, activation='relu', border_mode='same'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(p))

model.add(Convolution2D(nf,3,3, activation='relu', border_mode='same'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(p/2))
        
model.add(Convolution2D(1,3,3, border_mode='same'))

In [ ]:

model.compile(Adam(lr=0.00001), loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
trn_targets = np.array(trn_targets)
val_targets = np.array(val_targets)

trn_reshaped = trn_targets.reshape(trn_targets.shape[0],1,trn_targets.shape[1],trn_targets.shape[2])
val_reshaped = val_targets.reshape(val_targets.shape[0],1,val_targets.shape[1],val_targets.shape[2])

In [ ]:
model.fit(trn, trn_reshaped, batch_size=8, nb_epoch=5, 
             validation_data=(smp, val_reshaped))

In [ ]:
model.save_weights('mask_crop_fish_v2.pkl')

In [ ]:
def find_biggest_contour(contours):
    
    # nofish? 
    
    
#     if len(contours) == 1 :
#         return 0
    
    #if more than one contour, find the biggest contour surface
    biggest_contour_index = -1 
    biggest_contour_surface = 0

    i = 0
    for c in contours:
        # compute the center of the contour
        area = cv2.contourArea(c)
        if area > biggest_contour_surface:
            biggest_contour_surface = area
            biggest_contour_index = i
        i += 1
        
    return biggest_contour_index        

In [ ]:
test_path = DATA_HOME_DIR+"/test2/test_stg2/"
export_path = DATA_HOME_DIR+'/cropped_nof_excl2/'

crop_width = 800


In [ ]:
nb_test_samples = len(glob.glob('data/test2/*/*.jpg'))


test_datagen = ImageDataGenerator()
#         shear_range=0.2,
#         zoom_range=0.1,
#         rotation_range=10.,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         horizontal_flip=True)
    
    
test_generator = test_datagen.flow_from_directory(
            'data/test2/',
            target_size=size,
            batch_size=16,
            shuffle = False,
            classes = None,
            class_mode = None)

test_image_list = test_generator.filenames
predictions = model.predict_generator(test_generator, nb_test_samples)

In [ ]:
for i,t in enumerate(raw_test_filenames):
    
    
    if i < 289: #remove this to generate all crops
        continue
        
        
        
    test_img = cv2.imread(test_path+t)
    img_width = test_img.shape[1]
    img_height = test_img.shape[0]
    
#     plt.imshow(test_img)
    #predict
    inp = np.expand_dims(conv_test_feat[i], 0)
    conv = conv_fn([inp,0])[0, 0] #conf_fn shape (1, 1, 22, 40)
    cm = scipy.misc.imresize(conv, (img_height,img_width), interp='nearest')
    
    #find contours
    ret, thresh = cv2.threshold(cm, 200, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    #find center of the biggest contour

    i = find_biggest_contour(contours)
    M = cv2.moments(contours[i])
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    
    #calculate boundries of bounding box
    x = max(cX - crop_width/2, 0)
    y = max(cY - crop_width/2, 0)

    if x + crop_width > img_width:
        x = img_width - crop_width
    if y + crop_width > img_height:
        y = img_height - crop_width
        
    x = int(x)
    y = int(y)

    
###### plot bounding box    
#     cv2.rectangle(cm,(x,y),(x+crop_width,y+crop_width),color=(255, 255, 255))
#     plt.imshow(cv2.drawContours(cm, [contours[i]], -1, (0, 255, 0), 2))

##### crop the image and save
    crop_img = test_img[y:y+crop_width, x:x+crop_width]
#     plt.imshow(crop_img)
#     cv2.imwrite(export_path+t,crop_img)
    break



# Window based model:

In [ ]:
size = (360, 640)
model = VGG_16(size=size)


for layer in model.layers:
    layer.trainable = False
model.pop()
model.add(MaxPooling2D())
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

x_bb=Dense(4)(model.output)
x_class=Dense(8,activation='softmax')(model.output)
model=Model(model.input,[x_bb,x_class])

In [ ]:
model.compile(loss=['mse','categorical_crossentropy'],loss_weights=[.001,1.], optimizer=Adam(lr=0.00001), \
              metrics=["accuracy"])

In [ ]:
hist = model.fit(
        trn,
        [trn_bbox,trn_class],
        batch_size=16,
        nb_epoch=1,
        validation_data=(smp,[val_bbox,smp_class]),)

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['acc']); plt.plot(hist.history['val_acc']);
plt.title('model accuracy'); plt.ylabel('accuracy');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');

# summarize history for loss
plt.subplot(1, 2, 2)
plt.plot(hist.history['loss']); plt.plot(hist.history['val_loss']);
plt.title('model loss'); plt.ylabel('loss');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');
plt.show()

In [ ]:
model.save_weights('crop_fish_v2.pkl')

In [ ]:
pred = model.predict(smp[0:15])

In [ ]:
def show_bb_pred(i):
    bb = val_bbox[i]
    bb_pred = pred[0][i]
    plt.figure(figsize=(6,6))
    plot(smp[i])
    ax=plt.gca()
    ax.add_patch(create_rect(bb_pred, 'yellow'))
    ax.add_patch(create_rect(bb))

In [ ]:
# show first 15 results

for i in range(0,15):
    show_bb_pred(i)

In [ ]:
# crop result

crop_width=450
for i,t in enumerate(train_name_list):
    
    test_img = cv2.imread(t)
    img_width = test_img.shape[1]
    img_height = test_img.shape[0]
    
#     plt.imshow(test_img)
    #predict

    

    
    #find center of the biggest contour


    cX = int(bb_json[t.split('/')[-1]]['x']+bb_json[t.split('/')[-1]]['width']/2.)
    cY = int(bb_json[t.split('/')[-1]]['y']+bb_json[t.split('/')[-1]]['height']/2.)
    
    
    #calculate boundries of bounding box
    x = max(cX - crop_width/2, 0)
    y = max(cY - crop_width/2, 0)

#     if x + crop_width > img_width:
#         x = img_width - crop_width
#     if y + crop_width > img_height:
#         y = img_height - crop_width
        
    x = int(x)
    y = int(y)

    
###### plot bounding box    
#     cv2.rectangle(cm,(x,y),(x+crop_width,y+crop_width),color=(255, 255, 255))
#     plt.imshow(cv2.drawContours(cm, [contours[i]], -1, (0, 255, 0), 2))

##### crop the image and save
    crop_img = test_img[y:y+crop_width, x:x+crop_width]
#     plt.imshow(crop_img)
    cv2.imwrite(t,crop_img)
#     break